In [ ]:
import gc
import os
import json
import math
import shutil
from PIL import Image
import numpy as np

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torch.utils.data.dataloader import default_collate
import torchvision.models.detection as detection_models
import torchvision.models as models
import torchvision.transforms as T
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
torch.cuda.empty_cache()
gc.collect()


In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")


In [ ]:
in_dir = "/mnt/nis_lab_research/data/coco_files/clean/far_shah-b1-b2_cln"
out_dir = "../../data/obj_det/far_shah-b1-b2_cln"
batch_size = 8  
num_workers = 8 
shuffle = True
num_classes = 27 + 1


In [ ]:
def coco_train_test_split(in_dir, out_dir):
    fn = in_dir.split("/")[-1]
    
    if fn == None:
        fn = in_dir.split("/")[-2]
  
    # out_dir = os.getcwd() + "/" + fn + "_split"
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

        train_dir = out_dir + "/train"
        os.mkdir(train_dir)
        train_img_dir = train_dir + "/images"
        os.mkdir(train_img_dir)

        test_dir = out_dir + "/test"
        os.mkdir(test_dir)
        test_img_dir = test_dir + "/images"
        os.mkdir(test_img_dir)

        train_split = 0.8

        f = open(in_dir + "/result.json")
        coco_json = json.load(f)
        f.close()

        num_img = len(coco_json["images"])

        img_list = coco_json["images"]
        cat_list = coco_json["categories"]
        ann_list = coco_json["annotations"]

        train_num = math.floor(num_img * train_split)

        train_img_list = img_list[0:train_num]
        test_img_list = img_list[train_num:]

        for each in train_img_list:
            img_name = each["file_name"].split("/")[-1]
            shutil.copy(in_dir + "/images/" + img_name, train_img_dir + "/" + img_name)

        for each in test_img_list:
            img_name = each["file_name"].split("/")[-1]
            shutil.copy(in_dir + "/images/" + img_name, test_img_dir + "/" + img_name)

        co_val = train_img_list[-1]["id"]

        train_ann_list = []
        test_ann_list = []

        for each in ann_list:
            if each["image_id"] <= co_val:
                train_ann_list.append(each)
            else:
                test_ann_list.append(each)

        train_json = {
            "images": train_img_list,
            "categories": cat_list,
            "annotations": train_ann_list
        }

        test_json = {
            "images": test_img_list,
            "categories": cat_list,
            "annotations": test_ann_list
        }

        train_j_out = json.dumps(train_json, indent=4)
        test_j_out = json.dumps(test_json, indent=4)

        with open(train_dir + "/result.json", "w") as outfile:
            outfile.write(train_j_out)
        with open(test_dir + "/result.json", "w") as outfile:
            outfile.write(test_j_out)
            
        print("creating " + str(train_split) + " train test split to path: " + out_dir)
        
    else:
        print("directory: " + out_dir + " already exists!")

In [ ]:
coco_train_test_split(in_dir, out_dir)

In [ ]:
class CocoDetection(torch.utils.data.Dataset):
    def __init__(self, root, annFile, transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.transform = transform

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)

        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path)).convert('RGB')

        boxes = []
        labels = []
        img_ids = []
        for ann in annotations:
            # Convert COCO bbox format (x_min, y_min, width, height) to (x_min, y_min, x_max, y_max)
            x, y, w, h = ann['bbox']
            x_max = x + w
            y_max = y + h

            # Check if the bounding box is valid (positive width and height)
            if w > 0 and h > 0:
                boxes.append([x, y, x_max, y_max])
                labels.append(ann['category_id'])
                img_ids.append(img_id)

        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            img_ids.append(img_id)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            img_ids.append(img_id)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["img_ids"] = img_ids

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.ids)

In [ ]:
def cust_coll(batch):

    # Separate data and targets
    batch = list(zip(*batch))

    # Default collate for images
    images = default_collate(batch[0])

    # Targets are a list of dictionaries
    targets = batch[1]

    return images, targets

In [ ]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
train_set = CocoDetection(root="../../data/obj_det/far_shah-b1-b2_cln/train/images/", 
                          annFile="../../data/obj_det/far_shah-b1-b2_cln/train/result.json", 
                          transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=shuffle, 
                           num_workers=num_workers, collate_fn=cust_coll)

In [ ]:
test_set = CocoDetection(root="../../data/obj_det/far_shah-b1-b2_cln/test/images/", 
                         annFile="../../data/obj_det/far_shah-b1-b2_cln/test/result.json", 
                         transform=transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, 
                         num_workers=num_workers, collate_fn=cust_coll)

In [ ]:
def get_model(num_classes):
    # Load a pre-trained ResNet50 model
    backbone = models.resnet50(pretrained=True)
    
    # Remove the fully connected layers (classification head)
    # Keep only the layers up to the final convolutional layer
    modules = list(backbone.children())[:-2]
    backbone = torch.nn.Sequential(*modules)
    
    # Remove the fully connected layer (classification head)
    # Replace the classifier head of the backbone with a new one
    backbone.out_channels = 2048

    # Create an anchor generator for the FPN which is used in Faster R-CNN
    anchor_generator = AnchorGenerator(
        sizes=((32,), (64,), (128,), (256,), (512,)),  # One size for each feature map
        aspect_ratios=((0.5, 1.0, 2.0),) * 5  # Same aspect ratios for each feature map
    )

    # Define the Region Proposal Network (RPN)
    roi_pooler = MultiScaleRoIAlign(
        featmap_names=['0', '1', '2', '3'],
        output_size=7,
        sampling_ratio=2
    )

    # Create the Faster R-CNN model
    model = FasterRCNN(backbone,
                       num_classes=num_classes,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)

    return model

In [ ]:
# model = get_model(num_classes = 27+1)
# model.to(device)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretained=True)
num_classes = 27+1
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 3
for epoch in range(0, num_epochs):
    
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)

        # The loss is the sum of all individual losses
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
        
        print(i, running_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")


In [ ]:
# To load the model later
model = torch.load('./pth/far_shah-b1-b2_cln_rn50_pt_ep3.pth')
model.eval()  # Set it to evaluation mode

In [ ]:
for one in test_loader:
    for item in one[1]:
        print(item["img_ids"])

In [ ]:
# Put the model in evaluation mode
model.eval()

# Load the COCO ground truth
coco_path = "../../data/obj_det/far_shah-b1-b2_cln/test/result.json"
cocoGt = COCO(coco_path)

img_ids = []
with open(coco_path, "r") as f:
    obj = json.load(f)
for img in obj["images"]:
    img_ids.append(img["id"])


# Prepare for COCO evaluation
coco_results = []
ind = 0
with torch.no_grad():
    for images, targets in test_loader:
    
        images = list(img.to(device) for img in images)
        outputs = model(images)

        for i, output in enumerate(outputs):
            print(ind)
            img_ids = targets[i]["img_ids"]

            for box, label, score, img_id in zip(output["boxes"], output["labels"], output["scores"], img_ids):
                box = box.cpu().numpy()
                box = [float(n) for n in box]
                score = float(score)
                label = int(label)

                coco_result = {
                    "image_id": img_id,
                    "category_id": label,
                    "bbox": [box[0], box[1], box[2] - box[0], box[3] - box[1]],
                    "score": score
                }
                coco_results.append(coco_result)
                
            ind += 1

# Save the results in a file
with open("coco_results.json", "w") as f:
    json.dump(coco_results, f)

In [ ]:
# Load results into COCO data structure
cocoDt = cocoGt.loadRes("coco_results.json")

In [ ]:
# COCO evaluation
cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
cocoEval.params.imgIds = img_ids
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()